In [5]:
#Run this cell to get the output
import requests
import json
import csv
from bs4 import BeautifulSoup
import traceback
import cloudscraper
import pandas as pd
import numpy as np
import seaborn as sns
from decimal import Decimal
from re import sub
import matplotlib.pyplot as plt
from datetime import date
import os
import shutil

postcode_buy="MK36JS" 
radius_buy=10

postcode_sell="DT13GJ" 
radius_sell=25

#Remeber to check the maximum and minimum years allowed on the website 
min_year=2010
max_year=2015

min_price=500
max_price=2000

#State how many brands/ models you want to show in the final graphs 
no_brands = 50
no_models = 50

import argparse

In [17]:
def postcode_add(df,postcode):
    #Loop over the column names and add the postcode to each 
    for i in df.columns:
        concat_col = i + ' ' + postcode
        df = df.rename({i: concat_col.replace(' ','_')}, axis=1)

    return df

#function to retrieve cars based on parameters put in 
def retrieve_cars(postcode, radius, min_year, max_year, min_price, max_price,location_type):
    """
    Retrieve car listings based on specified search parameters.
    """
    cars = get_cars(
        postcode=postcode,
        radius=radius,
        min_year=min_year,
        max_year=max_year,
        min_price=min_price,
        max_price=max_price
    )
    # Clean up year and brand columns
    cars['year'] = cars['year'].dropna().apply(lambda x: int(str(x).split('(')[0]))
    cars['brand'] = cars['name'].apply(lambda x: x.split(' ')[0])
    cars['postcode'] = postcode
    cars = postcode_add(cars,location_type)
    
    return cars

def get_cars(
    make = "",
    model = "",
    postcode="SW1A 0AA", 
    radius=1500, 
    min_year=1995, 
    max_year=1995, 
    include_writeoff="exclude",
    verbose=False,
    min_price= 0,
    max_price = 99999):

    # To bypass Cloudflare protection
    scraper = cloudscraper.create_scraper()

    # Basic variables
    results = []
    n_this_year_results = 0

    url = "https://www.autotrader.co.uk/car-search?advertising-location=at_cars&exclude-delivery-option=on"

    keywords = {}
    keywords["mileage"] = ["miles"]
    keywords["transmission"] = ["Automatic", "Manual"]
    keywords["year"] = [" reg)"]
    keywords["engine"] = ["engine"]

    # Set up parameters for query to autotrader.co.uk
    params = {
        "sort": "relevance",
        "postcode": postcode,
        "price-from" : min_price,
        "price-to" : max_price,
        "radius": radius,
        "make": make,
        "model": model,
        "search-results-price-type": "total-price",
        "search-results-year": "select-year",
        "exclude-writeoff-categories": "on"
    }

    year = min_year
    page = 1
    attempt = 1
    
    try:
        while year <= max_year:
            params["year-from"] = year
            params["year-to"] = year
            params["page"] = page
            

            r = scraper.get(url, params=params)
            if verbose:
                print("Year:     ", year)
                print("Page:     ", page)
                print("Response: ", r)

            try:
                if r.status_code != 200:   # If not successful (e.g. due to bot protection)
                    print(r.status_code)

                    attempt = attempt + 1  # Log as an attempt

                    if attempt <= 5:
                        if verbose:
                            print("Exception. Starting attempt #", attempt, "and keeping at page #", page)
                    
                    else:
                        page = page + 1
                        attempt = 1
                        if verbose:
                            print("Exception. All attempts exhausted for this page. Skipping to next page #", page)

                if r.status_code == 404:
                    print(r.url)

                else:
                    j = r.content
                    
                    s = BeautifulSoup(j, features="html.parser")
                    
                    articles = s.find_all("article", attrs={"data-standout-type":""})

                    # If no results or reached end of results...
                    if len(articles) == 0 or r.url[r.url.find("page=")+5:] != str(page):
                        if verbose:
                            print("Found total", n_this_year_results, "results for year", year, "across", page-1, "pages")
                            if year+1 <= max_year:
                                print("Moving on to year", year + 1)
                                print("---------------------------------")

                        # Increment year and reset relevant variables
                        type(year)
                        year = year + 1
                        page = 1
                        attempt = 1
                        n_this_year_results = 0
                    else:
                        for article in articles:
                            car = {}
                            #car["name"] = article.find("h3", {"class": "product-card-details__title"}).text.strip()   
                            print(article.find("h3", {"class": "sc-dhKdcB fylIvF"}))          
                            car["link"] = "https://www.autotrader.co.uk" + \
                                    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
                                    .find("?")]
                            car["price"] = int(sub(r'[^\d.]', '', article.find("div", {"class": "product-card-pricing__price"}).text.strip()))

                            seller_info = article.find("ul", {"class": "product-card-seller-info__specs"}).text.strip()
                            car["seller"] = " ".join(seller_info.split())

                            key_specs_bs_list = article.find("ul", {"class": "listing-key-specs"}).find_all("li")

                            for key_spec_bs_li in key_specs_bs_list:

                                key_spec_bs = key_spec_bs_li.text

                                if any(keyword in key_spec_bs for keyword in keywords["mileage"]):
                                    car["mileage"] = int(key_spec_bs[:key_spec_bs.find(" miles")].replace(",",""))
                                elif any(keyword in key_spec_bs for keyword in keywords["BHP"]):
                                    car["BHP"] = int(key_spec_bs[:key_spec_bs.find("BHP")])
                                elif any(keyword in key_spec_bs for keyword in keywords["transmission"]):
                                    car["transmission"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["fuel"]):
                                    car["fuel"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["owners"]):
                                    car["owners"] = int(key_spec_bs[:key_spec_bs.find(" owners")])
                                elif any(keyword in key_spec_bs for keyword in keywords["body"]):
                                    car["body"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["ULEZ"]):
                                    car["ULEZ"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["year"]):
                                    car["year"] = key_spec_bs
                                elif key_spec_bs[1] == "." and key_spec_bs[3] == "L":
                                    car["engine"] = float(sub(r'[^\d.]', '', key_spec_bs))

                            results.append(car)
                            n_this_year_results = n_this_year_results + 1

                        page = page + 1
                        attempt = 1

                        if verbose:
                            print("Car count: ", len(results))
                            print("---------------------------------")

            except KeyboardInterrupt:
                break

            except:
                traceback.print_exc()
                attempt = attempt + 1
                if attempt <= 5:
                    if verbose:
                        print("Exception. Starting attempt #", attempt, "and keeping at page #", page)
                else:
                    page = page + 1
                    attempt = 1
                    if verbose:
                        print("Exception. All attempts exhausted for this page. Skipping to next page #", page)

    except KeyboardInterrupt:
        pass


    return pd.DataFrame(results)

In [18]:
location_buy = retrieve_cars(
                           postcode = postcode_buy,
                           radius = radius_buy ,
                           min_year = min_year ,
                           max_year = max_year ,
                           min_price = min_price,
                           max_price = max_price,
                           location_type = 'buy')

None
None
None


Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~

None
None
None


Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~

None
None


...

None
None


Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable


None
None
None


Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\ngben\AppData\Local\Temp\ipykernel_14396\3348428479.py", line 134, in get_cars
    article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
    ~~~~~~~~~~~~~~~

KeyError: 'year'